In [1]:
import pandas as pd
import numpy as np

# Load Dataset

In [22]:
# ctrl = pd.read_csv('new_ctrl.csv')
# gpt = pd.read_csv('new_gpt.csv')
# gpt2 = pd.read_csv('new_gpt2.csv')
gpt3 = pd.read_csv('reddit_data_gpt3.csv')
instructgpt = pd.read_csv('reddit_data_instructgpt.csv')[['Generation', 'label']]
# grover = pd.read_csv('new_grover.csv')
# xlm = pd.read_csv('new_xlm.csv')
# xlnet = pd.read_csv('new_xlnet.csv')

# pplm = pd.read_csv('new_pplm.csv')
human = pd.read_csv('reddit_data_human.csv')[['Generation', 'label']]
# fair = pd.read_csv('new_fair.csv')

In [23]:
print(gpt3.shape, instructgpt.shape, human.shape)
gpt3.dropna(inplace=True)
instructgpt.dropna(inplace=True)
human.dropna(inplace=True)
print(gpt3.shape, instructgpt.shape, human.shape)

(507, 2) (533, 2) (508, 2)
(506, 2) (475, 2) (505, 2)


In [26]:
gpt3 = gpt3.sample(475)
instructgpt = instructgpt.sample(475)
human = human.sample(475)

print(gpt3.shape, instructgpt.shape, human.shape)

(475, 2) (475, 2) (475, 2)


# Data Preparation 

In [28]:
random_binary_t1 = pd.concat([human,human, human, gpt3, gpt3, gpt3, instructgpt, instructgpt, instructgpt])

In [27]:
random_binary_t2 = pd.concat([human.sample(475),
                              gpt3.sample(475),
                              instructgpt.sample(475),
                              human.sample(475),
                              gpt3.sample(475),
                              instructgpt.sample(475),
                              gpt3.sample(475),
                              instructgpt.sample(475),
                              human.sample(475)])


In [29]:
random_binary_t1['label'].value_counts()

human          1425
gpt3           1425
instructgpt    1425
Name: label, dtype: int64

In [30]:
random_binary_t2['label'].value_counts()

human          1425
gpt3           1425
instructgpt    1425
Name: label, dtype: int64

In [31]:
human.shape

(475, 2)

In [32]:
len(random_binary_t1) == len(random_binary_t2)

True

In [33]:
len(random_binary_t1)

4275

In [34]:
random_binary_t2.head()

,Generation,label
289,"Well, it's a change-my-view about changing vie...",human
167,Many (if not all) of the same states (with som...,human
432,This week there will be a vote to decide wheth...,human
70,"So they don’t actually know what QAnon is, but...",human
188,If intelligence can confirm Putin and his most...,human


In [35]:
new_label = [np.ones(len(human)), 
             np.zeros(2 * 475), 
             np.ones(len(instructgpt)), 
             np.zeros(4 * 475), 
             np.ones(len(gpt3)) 
            ]


In [36]:
flat_list = [item for sublist in new_label for item in sublist]



In [37]:
len(flat_list)

4275

In [38]:
len(random_binary_t1)

4275

In [39]:
len(random_binary_t2)

4275

In [40]:
binary_random = pd.DataFrame({'T1': list(random_binary_t1['Generation']), 'T2': list(random_binary_t2['Generation']), 'class': flat_list})

In [41]:
binary_random.to_csv('binary_random.csv')

# Create Balanced Dataset

In [51]:
(133*6) + (134 * 2)

1066

In [53]:
237 + 238

475

In [52]:
475/2

237.5

In [54]:
balance_T1 = pd.concat([human.sample(475), gpt3.sample(237), instructgpt.sample(238), gpt3.sample(237), instructgpt.sample(238), human.sample(475)])
balance_T2 = pd.concat([human, human, gpt3.sample(237), instructgpt.sample(238), gpt3.sample(237), instructgpt.sample(238)])


In [55]:
balance_T2['label'].value_counts()

human          950
instructgpt    476
gpt3           474
Name: label, dtype: int64

In [56]:
balance_T1['label'].value_counts()

human          950
instructgpt    476
gpt3           474
Name: label, dtype: int64

In [57]:
balance_label = [np.ones(475),
                 np.zeros(475), 
                 np.ones(475), 
                np.zeros(475)]

In [58]:
balance_label = [item for sublist in balance_label for item in sublist]


In [59]:
balanced_p1 = pd.DataFrame({'T1': list(balance_T1['Generation']),
                            'T2': list(balance_T2['Generation']),
                            'class': balance_label})


In [60]:
balanced_p1.to_csv('balanced_p1.csv')

# Random Binary Classification 

In [61]:

import numpy as np
import pandas as pd 
from sklearn import metrics
import xgboost as xgb
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, f1_score
from sklearn import metrics

In [62]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix


In [63]:
from yellowbrick.classifier import ClassPredictionError
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score

In [64]:
balanced_p1.dropna(inplace=True)

In [65]:
balanced_p1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1900 entries, 0 to 1899
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   T1      1900 non-null   object 
 1   T2      1900 non-null   object 
 2   class   1900 non-null   float64
dtypes: float64(1), object(2)
memory usage: 44.7+ KB


In [66]:
def classify(data, label):
            
    data = data['T1'] + data['T2']
        
    X_train, X_test, y_train, y_test = train_test_split(data, label, stratify = label, test_size = 0.2, random_state = 1234)
    
    clf = xgb.XGBClassifier()    
    v = TfidfVectorizer()
    
    train_corpus = X_train
    test_corpus = X_test
    
    train_vector = v.fit_transform(train_corpus)
    test_vector = v.transform(test_corpus)
    
    
    fit = clf.fit(train_vector,y_train)
    pred = clf.predict(test_vector)
    
    
    Accuracy = accuracy_score(y_test,pred)
    F1 = f1_score(y_test, pred, average='macro')
    print("Accuracy:", Accuracy)
    print('F1:', F1)
    
    rec = recall_score(y_test, pred, average='macro')
    print('Recall: ', rec)
    prec = precision_score(y_test, pred, average='macro')
    print('Precision: ', prec)
    
    
    return y_test, pred

In [67]:
classify(balanced_p1[['T1', 'T2']], balanced_p1['class'])

Accuracy: 0.7684210526315789
F1: 0.7681898846495121
Recall:  0.7684210526315789
Precision:  0.7694960507286683


(377     1.0
 1713    0.0
 1695    0.0
 1496    0.0
 1340    1.0
        ... 
 1698    0.0
 1765    0.0
 1607    0.0
 1129    1.0
 1623    0.0
 Name: class, Length: 380, dtype: float64,
 array([0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1,
        0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0,
        0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0,
        0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0,
        0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0,
        1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1,
        1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1,
        0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0,
        0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0,


In [68]:
binary_random.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4275 entries, 0 to 4274
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   T1      4275 non-null   object 
 1   T2      4275 non-null   object 
 2   class   4275 non-null   float64
dtypes: float64(1), object(2)
memory usage: 100.3+ KB


In [69]:
binary_random.dropna(inplace=True)

In [70]:
classify(binary_random[['T1', 'T2']], binary_random['class'])

Accuracy: 0.6666666666666666
F1: 0.61986661986662
Recall:  0.618421052631579
Precision:  0.6217948717948718


(530     0.0
 3977    1.0
 4167    1.0
 992     0.0
 3118    0.0
        ... 
 3908    1.0
 2228    0.0
 620     0.0
 1138    0.0
 1952    0.0
 Name: class, Length: 855, dtype: float64,
 array([1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1,
        0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0,
        1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0,
        0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1,
        0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0,
        0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1,
        0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1,
        0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0,
        0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0,


In [71]:
balanced_p1 

,T1,T2,class
0,I am not for or against any of the ideologies....,Child poverty causes a number of problems for ...,1.0
1,The Green and Libertarian parties have complet...,I am German and my opinion is based on the fla...,1.0
2,"As a Libertarian, I'm a firm believer that eve...","Today is Australia's National Holiday, celebra...",1.0
3,I am a young person (&lt;30 years) who has bee...,"Abortion, gun safety laws, school shootings, r...",1.0
4,"Let me start by saying, I was once just like y...","Now, he's a great guy, but politically... let'...",1.0
...,...,...,...
1895,Obviously the trade was made because it was th...,\n\nCMV: No. People have the right to have the...,0.0
1896,"For Context, FIFA's national eligibility rule...",\n\nCMV: Agreed. Both sides of the political s...,0.0
1897,This takes a lot of backstory. so buckle up.\n...,\n\n1. Vermont\n2. Oregon\n3. Massachusetts\n4...,0.0
1898,I lived in Ukraine and they wholeheartedly bel...,\n\nThat is perfectly understandable. Politics...,0.0


Reddit Task P1

balanced_p1 -
Accuracy: 0.7684210526315789
F1: 0.7681898846495121
Recall:  0.7684210526315789
Precision:  0.7694960507286683
    
Binary Random
Accuracy: 0.6666666666666666
F1: 0.61986661986662
Recall:  0.618421052631579
Precision:  0.6217948717948718